# 第六章：预知（逐步思考）

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Lesson

If someone woke you up and immediately started asking you several complicated questions that you had to respond to right away, how would you do? Probably not as good as if you were given time to **think through your answer first**. 

Guess what? Claude is the same way.

**Giving Claude time to think step by step sometimes makes Claude more accurate**, particularly for complex tasks. However, **thinking only counts when it's out loud**. You cannot ask Claude to think but output only the answer - in this case, no thinking has actually occurred.

### Examples

在下面的提示中，对于人类读者来说，很明显第二句话与第一句话是矛盾的。但**Claude对"无关"这个词的理解过于字面化**。

In [ ]:
# 提示
# 这个例子展示了Claude有时过于字面化理解文本的问题
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# 打印Claude的响应
print(get_completion(PROMPT))

为了改进Claude的响应，让我们**允许Claude在回答之前先思考**。我们通过字面上明确说明Claude应该采取的步骤来处理和思考其任务来做到这一点。结合一点角色提示，这使Claude能够更深入地理解评论。

In [ ]:
# 系统提示
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# 提示
# 这个改进版本让Claude在给出最终答案前先考虑正反两方面的论证
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# 打印Claude的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claude有时对顺序很敏感**。这个例子处于Claude理解细致入微文本能力的前沿，当我们交换上一个例子中论证的顺序，使消极论证在前，积极论证在后时，这会改变Claude的整体评估为积极。

在大多数情况下（但令人困惑的是，并非全部），**Claude更倾向于选择两个选项中的第二个**，可能是因为在其来自网络的训练数据中，第二个选项更可能是正确的。

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

**让Claude思考可以将Claude的答案从错误转变为正确**。在许多Claude犯错的情况下就是这么简单！

让我们来看一个Claude答案不正确的例子，看看要求Claude思考如何解决这个问题。

In [ ]:
# 提示
# 这个例子展示Claude在没有逐步思考时可能给出错误答案
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# 打印Claude的响应
print(get_completion(PROMPT))

让我们通过要求Claude逐步思考来解决这个问题，这次使用`<brainstorm>`标签。

In [ ]:
# 提示
# 改进版本：要求Claude先在<brainstorm>标签中思考，然后给出答案
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# 打印Claude的响应
print(get_completion(PROMPT))

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## 练习
- [练习 6.1 - 电子邮件分类](#exercise-61---classifying-emails)
- [练习 6.2 - 电子邮件分类格式化](#exercise-62---email-classification-formatting)

### 练习 6.1 - 电子邮件分类
在本练习中，我们将指导Claude将电子邮件分类到以下类别中：										
- (A) 售前问题
- (B) 损坏或有缺陷的商品
- (C) 账单问题
- (D) 其他（请说明）

在练习的第一部分，修改`PROMPT`使**Claude输出正确的分类且仅输出分类**。您的答案需要**包括正确选择的字母（A-D），带括号，以及类别名称**。

参考`EMAILS`列表中每封邮件旁边的注释，了解该邮件应该被分类到哪个类别。

In [ ]:
# 带有占位符的提示模板
# 需要修改这个提示来正确分类电子邮件
PROMPT = """Please classify this email as either green or blue: {email}"""

# Claude响应的预填充（如果有的话）
PREFILL = ""

# 存储为列表的可变内容
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) 损坏或有缺陷的商品
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) 售前问题 或 (D) 其他（请说明）
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) 账单问题
    "How did I get here I am not good with computer.  Halp." # (D) 其他（请说明）
]

# 正确分类存储为列表的列表，以适应每封邮件可能有多个正确分类的情况
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 用于正则表达式评分的每个类别的字符串值字典
REGEX_CATEGORIES = {
    "A": "A\) P",  # (A) P 匹配 "(A) Pre-sale question" 的开头
    "B": "B\) B",  # (B) B 匹配 "(B) Broken or defective item" 的开头  
    "C": "C\) B",  # (C) B 匹配 "(C) Billing question" 的开头
    "D": "D\) O"   # (D) O 匹配 "(D) Other" 的开头
}

# 遍历电子邮件列表
for i,email in enumerate(EMAILS):
    
    # 将电子邮件文本替换到电子邮件占位符变量中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取Claude的响应
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 对Claude的响应进行评分
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # 打印Claude的响应
    print("--------------------------- 变量替换后的完整提示 ---------------------------")
    print("用户轮次")
    print(formatted_prompt)
    print("\n助手轮次")
    print(PREFILL)
    print("\n------------------------------------- Claude的响应 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 评分 ------------------------------------------")
    print("本练习已正确解决:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

Still stuck? Run the cell below for an example solution.						

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### 练习 6.2 - 电子邮件分类格式化
在本练习中，我们将完善上述提示的输出，以产生我们想要的确切格式的答案。

使用您最喜欢的输出格式化技术，让Claude仅将正确分类的字母包装在`<answer></answer>`标签中。例如，第一封邮件的答案应该包含确切的字符串`<answer>B</answer>`。

如果您忘记了每封邮件对应的正确字母类别，请参考`EMAILS`列表中每封邮件旁边的注释。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))